In [2]:
import os
import sys
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from pathlib import Path
import random

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib qt

#### Load EEGexp object

In [5]:
rec_folder = r'F:\EEG_exp\mouse543396\estim_vis1_autosort\experiment1\recording1' # points to folder containing stim_table.csv
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation
SomnoSuite log file not found.


In [6]:
mask_probe = 'probeC'

In [7]:
# make paths
stim_log_path = exp.stimulus_log_file
probe_info = exp.ephys_params[mask_probe]['probe_info']
ap_time_path = exp.ephys_params[mask_probe]['ap_timestamps']
# spike_times
spike_times_path = exp.ephys_params[mask_probe]['spike_times']
spike_clusters_path = exp.ephys_params[mask_probe]['spike_clusters']

# paths for artifact (this code creates these files at the end, not necessary to have them, but should specify where you want them saved)
arttimes_path = os.path.join(exp.experiment_folder, (mask_probe + '_sorted'), 'artifact_onset_master_clock.npy')
maskdur_path = os.path.join(exp.experiment_folder, (mask_probe + '_sorted'), 'mask_sampleduration.npy')

### Align spike_times to master clock

In [8]:
# load AP master timestamps and spike_times (which is in samples)
APmaster_time = np.load(ap_time_path)
spike_times = np.load(spike_times_path.replace('_master_clock', ''))

In [1]:
spike_times_master = APmaster_time[spike_times]

NameError: name 'APmaster_time' is not defined

In [10]:
print(len(spike_times_master))
print(spike_times_master[0:5])
print(spike_times_master[-5:])

3566769
[20.7764865  20.77821984 20.77871984 20.77975317 20.78091984]
[3227.82743578 3227.82786911 3227.82830244 3227.82863578 3227.83050245]


### Mask spike times during artifact

In [8]:
# load files
master_spike_times = np.load(spike_times_path)
spike_clusters = np.load(spike_clusters_path)

artifact_onset = np.load(arttimes_path)
mask_duration = np.load(maskdur_path)

In [9]:
print(master_spike_times.shape)
print(spike_clusters.shape)
print(artifact_onset.shape)
print(artifact_onset[0:5])
print(mask_duration)

(3566769,)
(3566769,)
(300,)
[33.63932953 37.74173262 41.50513545 45.65963859 49.84927508]
60


In [10]:
np.sum(np.isnan(master_spike_times))

0

In [11]:
np.sum(np.isnan(artifact_onset))

0

#### Goal: identify any spike_times that occur during the masked artifact times (these spikes should be ignored). Once identified the spikes will be associated with a distinct cluster id, to be flagged as noise during phy clustering.

In [12]:
print(spike_clusters.max())

365


In [13]:
print(spike_clusters.dtype)

uint32


In [14]:
print(mask_duration/30000)

0.002


In [15]:
mask_spikes = np.zeros((0,), dtype='int')
for ind, arton in enumerate(artifact_onset):
    if np.isnan(arton):
        continue
    else:
        art_spikes = np.where((master_spike_times > arton) & (master_spike_times < (arton + (mask_duration/30000))))[0]
        mask_spikes = np.concatenate((mask_spikes, art_spikes))

In [16]:
print(mask_spikes.shape)
print(mask_spikes[0:10])

(446,)
[33808 33809 33810 43238 43239 51846 51847 62359 62360 62361]


#### With artifact mask associated spikes identified, now change their cluster id to next highest cluster number.

In [17]:
spike_clusters[mask_spikes] = spike_clusters.max()+1

#### Now save updated spike clusters

In [18]:
np.save(spike_clusters_path, spike_clusters)